In [1]:
!mkdir /local/shared/pinsky_lab/sequencing/hiseq_2017_06_14_SEQ22/

In [ ]:
wget -r https://htseq.princeton.edu/tmp/cJ30KyZbQpRkVVaXO/ -P /local/shared/pinsky_lab/sequencing/hiseq_2017_06_14_SEQ22/ --no-check-certificate

In [1]:
!mkdir /local/home/sarahgw/WGS/Sequences/Pool6/

In [ ]:
scp Pool6_Barcodes.txt sarahgw@amphiprion.deenr.rutgers.edu:/local/home/sarahgw/WGS/Sequences/Pool6/

In [ ]:
mkdir $(awk '{print $1}' /local/home/sarahgw/WGS/Sequences/Pool6/Pool6_Barcodes.txt)

In [2]:
!ls /local/home/sarahgw/WGS/Sequences/Pool6/

NJ_E_05  NJ_E_10  NY_U_27  Pool6_Barcodes.txt  VT_E_21	VT_U_20  VT_U_26
NJ_E_06  NJ_E_11  NY_U_28  VT_E_18	       VT_E_22	VT_U_21
NJ_E_07  NY_U_25  NY_U_29  VT_E_19	       VT_E_23	VT_U_22
NJ_E_08  NY_U_26  NY_U_30  VT_E_20	       VT_E_24	VT_U_25


In [4]:
!rmdir /local/home/sarahgw/WGS/Sequences/Pool6/NJ_E_11 /local/home/sarahgw/WGS/Sequences/Pool6/VT_E_23 /local/home/sarahgw/WGS/Sequences/Pool6/VT_U_21

In [5]:
!ls /local/home/sarahgw/WGS/Sequences/Pool6/

NJ_E_05  NJ_E_10  NY_U_28	      VT_E_18  VT_E_22	VT_U_25
NJ_E_06  NY_U_25  NY_U_29	      VT_E_19  VT_E_24	VT_U_26
NJ_E_07  NY_U_26  NY_U_30	      VT_E_20  VT_U_20
NJ_E_08  NY_U_27  Pool6_Barcodes.txt  VT_E_21  VT_U_22


In [6]:
rm /local/home/sarahgw/WGS/Sequences/Pool6/Pool6_Barcodes.txt

move files into directories

In [ ]:
for dir in $(ls); do cd $dir; cp /local/shared/pinsky_lab/sequencing/hiseq_2017_06_14_SEQ22/"$dir"_Demultiplexed-fastq-fileset-sample-Pool-6-from-flowcell-HM327BCXY-on-2017-06-07_Read_1_passed_filter.fastq.gz /local/shared/pinsky_lab/sequencing/hiseq_2017_06_14_SEQ22/"$dir"_Demultiplexed-fastq-fileset-sample-Pool-6-from-flowcell-HM327BCXY-on-2017-06-07_Read_4_passed_filter.fastq.gz  . ; cd ..; done

Run bioinformatics

In [ ]:
~/Notebooks/WNS_WGS/Bioinfo_pipeline.sh 

In [1]:
!tail -n 21 /local/home/sarahgw/WGS/Sequences/Metrics/Pool5_reads

18477565
38334353
49017272
8083841
17155455
15667161
15572640
11903528
5938826
5639557
8781465
13566455
14086624
12512138
4589460
18786534
8031030
12273895
8839475
17312151
10179970


In [ ]:
rm */*passed_filter.fastq
    rm */*FU1.fastq
    rm */*FU2.fastq
    rm */*trim_p.fastq
    rm */*trim_up.fastq
    
    find . -name "*.tagged.fastq" -exec mv -i {} ../Pool6Contaminants/ \;
    for file in $(ls); do tar -zcvf "$file".tar.gz "$file" ; done
    rm *.fastq
    mkdir /Scratch/sarahgw/Pool4Contaminants/
    mv *.gz /Scratch/sarahgw/Pool4Contaminants/

Map reads, sort, remove duplicates

In [ ]:
for dir in $(ls); do cd $dir; /local/home/sarahgw/anaconda2/bin/bowtie2 --very-sensitive-local -p 12 -x /local/home/sarahgw/references/Myotis_Lucifugus2.0/MyoLuc_toplevel_index -1 $(pwd)/"$dir"_FU1_trim_p.tagged_pfilter.fastq -2 $(pwd)/"$dir"_FU2_trim_p.tagged_pfilter.fastq -U $(pwd)/"$dir"_FU1_trim_up.tagged_filter.fastq,$(pwd)/"$dir"_FU2_trim_up.tagged_filter.fastq -S $(pwd)/"$dir"_pairedplus_eg2.sam; cd ..; done

In [ ]:
for dir in $(ls); do cd $dir; java -jar /local/home/sarahgw/picard.jar SortSam \
      I=$(pwd)/"$dir"_pairedplus_eg2.sam \
      O=$(pwd)/"$dir"_pairedplus_eg2_sorted.bam \
      SORT_ORDER=coordinate; cd ..; done

In [ ]:
for dir in $(ls); do cd $dir; java -jar /local/home/sarahgw/picard.jar MarkDuplicates \
      I=$(pwd)/"$dir"_pairedplus_eg2_sorted.bam \
      O=$(pwd)/"$dir"_pairedplus_marked_dupes.bam \
      M=$(pwd)/"$dir"_pairedplus_marked_dup_metrics.txt \
      REMOVE_DUPLICATES=true \
      ASSUME_SORT_ORDER=coordinate; cd ..; done 

In [ ]:
for dir in $(ls); do cd $dir; samtools view -b $(pwd)/"$dir"_pairedplus_marked_dupes.bam | genomeCoverageBed -ibam stdin -g /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myoluc_genomeFile.txt>$(pwd)/"$dir"_Coverage.txt; cd ..; done

In [ ]:
for dir in $(ls); do cd $dir;  sed "s/$/\t"$dir"/" "$dir"_Coverage.txt > "$dir"_Coverage_name.txt ; cd ..; done